# Homework 4 Monte Carlo Integration and Simulations


## Monte Carlo Integration 

A sphere of radius $r_1$ consists of two different materials, with densities $\rho_1$ and $\rho_2$. The material with density $\rho_2$ is located within a cylinder of radius $r_2$, as illustrated in the figure, and the material of density $\rho_1$ fills up the rest of the sphere. Write a program using Monte Carlo integration to calculate the two moments of inertia of this sphere corresponding to rotation about the $z$ and $x$ axis. The inner cylinder is centered around the $z$-axis, as also shown in the figure.

<img src="HW4_Fig1.jpg" width="400"  alt="Sphere with a hole">


Carry out the calculation using Monte Carlo sampling of the moment of inertia integral

$$
I=\int d x \int d y \int d z \rho(x, y, z) r_{\perp}^2(x, y, z)
$$
where $r_{\perp}(x, y, z)$ is the perpendicular distance of the point $(x, y, z)$ from the axis of rotation (here the $x$ or $z$ axis, giving $I_x$ and $I_z$ ). Enclose the sphere in a box with side $L=2 r_1$ in order to easily do the calculation using $(x, y, z)$ points. 

The program should read the following input from a file `read.in`:

where `r1`, `r2` are the two radii (in m ), `rho1`, `rho2` are the densities (in {kg} / {m}$^3$ ), `npt` is the number of random points generated per "bin" (for which bin averages are computed) and `nbi` is the number of bins (on the basis of which the final average and statistical error are computed).

Bin averages should be computed for both the $I_z$ and $I_x$ moments of inertia and these should be writen to a file `bin.dat` containing `nbi` lines, each with the bin number followed by the `I_z` and `I_x` values (write these averages to the file after each bin is completed; it is not necessary to store the data in the program). 

The final average and error bar (standard deviation of the mean) computed using the bin averages should be written to a file `res.dat`.

As a specific case, do the calculation for a copper (8930 kg/m3) sphere of radius 5 cm with an inner gold (19320 kg/m3) cylinder of radius 1 cm. Use $10^6$ points per bin (`npt`) and do the calculation for `nbi`=50,500,5000. For the final case, construct histograms of the bin averages of the two moments of inertia (with the width of the histogram bins chosen in a reasonable way to get of the order tens of histogram bins with significant weight). 

The report on this problem needs to contain only the final numerical results (averages and standard deviations) for the three runs and the histograms for the last run.

In [ ]:
### Your Code Here 

## Two-dimensional Ising model

### The Ising model

The Ising model is one of the drosophila of physics. At first look, it’s a simple model of crystalline materials which attributes magnetism to the orientation of magnetic moments on a lattice, summarized by the Hamiltonian or energy
$$
E=-\sum_{i, j} J_{i j} s_i s_j+\sum_i h_i s_i.
$$

The variables $s_i$ express the possible orientations for each moment, while the entries $J_{i j}$ of the (symmetric) interaction matrix characterize the interaction energy of moments $i$ and $j$. The value $h_i$ specifies a magnetic field on each site $i$.

Ising's original formulation (often referred to as the Ising model) considers moments with two orientations, $s_i= \pm 1$, which only interact with their nearest neighbors. Below we will call this the two-dimensional Ising model.

A configuration of an Ising model is a specification of every moment's orientation - an assignment of values to the $s_i$ variable for all moments in the system. For our purposes, configurations are 'snapshots' of the system which our simulator will hold in memory, like a vector containing $\pm 1$ in its $i$ th entry to specify whether moment $i$ points up or down.

### Statistical Mechanics 
There are a couple facts we need to know about statistical mechanics.
1. We can specify configurations of our system $c$. In the case of an Ising model, the configuration is the orientation of each spin.
2. Each configuration $c$ has an energy $E(c)$. We've already seen this for the Ising model.
3. For classical systems in contact with an energy reservoir of temperature $T$, the probability $P$ of finding the system in configuration $c$ is

$$
P(c)=\frac{e^{-\beta E(c)}}{\sum_c e^{-\beta E(c)}}
$$

where $E(c)$ is the configuration's energy and $\beta \equiv 1 /(k T)$ with $k$ Boltzmann's constant and $T$ is the temperature. We will work in units where $k=1$.

### Markov Chains
We have just learned that statistical mechanics tells us that if we look at the system, each configuration $c$ appears with probability $P(c) \sim \exp (-\beta E(c))$. Our first goal is to implement an algorithm which gives us configurations $c$ with probability $P(c)$.

One such algorithm is markov chain Monte Carlo(MCMC).

A Markov chain is a process where you are at some state $c$ (i.e. a configuration) and then you choose another state $c^{\prime}$ with a probability that only depends on $c$. In other words, you can't use the fact that previously you were at configuration $b$ to decide the probability $P\left(c \rightarrow c^{\prime}\right)$ you're going to go from $c$ to $c^{\prime}$. This process is called a memoryless process.

As long as you do a (non-pathalogical) random walk in a memoryless way, you're guaranteed that, after walking around enough, that the probability you are in configuration $c$ is some $\pi(c) . \pi$ is called the stationary distribution. Different markov chains will have different stationary distributions. A Markov chain is nonpathological if:
- It is aperiodic; it doesn't cycle between configurations in a subset of the system's full configuration space.
- It is connected; given two configurations, there is a path with non-zero probability that the chain could (but not necessarily will) follow to get from one to the other.

To simulate the Ising model, we wish to build a markov chain which has the stationary distribution $\pi(c) \sim \exp \left(-\beta E_c\right)$. We will do so with a very famous algorithm which lets us build a Markov chain for any $\pi(c)$ we want.

### The Metropolis-Hastings algorithm

If you know your desired stationary distribution, the Metropolis-Hastings algorithm provides a canonical way to generate a Markov chain that produces it.  The steps of the algorithm are:
- Start with some configuration $c$
- Propose a move to a new trial configuration $c^{\prime}$ with a transition probability $T\left(c \rightarrow c^{\prime}\right)$
- Accept the move to $c^{\prime}$ with probability $\min \left(1, \frac{\pi\left(c^{\prime}\right)}{\pi(c)} \frac{T\left(c^{\prime} \rightarrow c\right)}{T\left(c \rightarrow c^{\prime}\right)}\right)$

### States and parameters
For our simulation, we are going to work on a two-dimensional lattice, have no magnetic field ( $h_i=0$ ) and set $J_{i j}=J$ for all $i$ and $j$ being nearest neighbors (and zero otherwise). This leaves us with an energy

$$
E=-J \sum_{(x, y)} s_{(x, y)} s_{(x+a, y+b)}
$$

where $(a, b) \in\{(1,0),(0,1),(-1,0),(0,-1)\}$
For this problem we will be working on a $L \times L$ square grid with the periodic boundary condition, i.e. 
$s_{(x+L, y)}=s_{(x, y)}$,$s_{(x, y+L)}=s_{(x, y)}$`. We are going to represent the state of our spins by a $L \times L$ array which can have either a -1 (spin-down) or 1 (spin-up). 



In [ ]:
import numpy as np


### We always use a seed for random number generators, so that your runs are reproducible
### Notice that the default numpy random number generator is PCG-64 https://www.pcg-random.org
seed=42 
np.random.seed(seed)

In [ ]:
### Initialize the configuration with random +1 or -1
def init_configuration(L):
    return np.random.choice([-1,1], (L,L))



### Computing the Energy and Magnetization
The next step is how to compute the energy $E(c)$ of a configuration. Write some code (i.e `def Energy (conf)`) which allows you to do this.

In addition to computing the energy for a single configuration, it's going to be critical to be able to compute the **energy difference** between two configurations that differ by a single spin flip (i.e. `def deltaE(spin_to_flip)`)

Why is this? Because the acceptance criterion we are going to use is the ratio of $\exp \left[-\beta E_{\text {new }}\right] / \exp \left[-\beta E_{\text {old }}\right]=\exp [-\beta \Delta E]$.

We also need to compute the magnetization per site. Notice that we are performing a finite-size simulation so there will be no symmetry breaking, i.e. a naive sum of total spins will average to zero. You need to tweek your estimator for the magnetization. 

In [ ]:
### energy per site
def Energy(config):


    return

### energy difference when flipping a spin
def deltaE(spin_to_flip):


    return


### magnetization per site
def Mag(config):

    return

### magnetization difference when flipping a spin
def deltaM(spin_to_flip):

    return

## Main simulation loop

- Read from `config.ini` the simulation parameters: `L`, `thermalization_sweeps`, `num_bins` and `num_sweeps`. 
- Perform the simulation from a random configuration and run `thermalization_sweeps` of sweeps for the system to reach equlibrium. 
- Save the spin configuration and seed.
- Run `num_bins` simulations with `num_sweeps` in each bin. Save the spin configuration and seed after each bin. Output $\langle E \rangle$, $\langle E^2 \rangle$, $\langle |m| \rangle$, and  $\langle m^2 \rangle$ to a file `ising.dat`
- Perform data analysis using the data in `ising.dat`


In [ ]:
### Main loop

with open('config.ini', 'r') as f:
    L = int(f.readline().strip())
    thermalization_sweeps = int(f.readline().strip())
    num_bins = int(f.readline().strip())
    num_sweeps = int(f.readline().strip())

num_sites = L * L
config=init_configuration(L)


for sweep in range(thermalization_sweeps):
    for step in range(num_sites):
        #flip spins


### compute the energy and magnetization once the system has thermalized
ene=Energy(config)
mag=Mag(config)
for bin in range(num_bins):
    for sweep in range(num_sweeps):
        for step in range(num_sites):
            #flip spins




        # measure energy and magnetization 
      